**Prepare data for training**

My idea was the network would be trained to decide which factor is important, which is not, just feed all available features into the network


* Only continuous and 1/0 data

* Extract TITLE from NAME, idea from another Discussion

* 1 for alive, -1 for dead

* Create feature for the same Ticket number

In [1]:
# import re
# temp = "Hoyt, Mr. Frederick Maxfield"
# print(re.search(' ([A-Za-z]+)\.', temp))

import pandas as pd
import numpy as np

temp_df = pd.read_csv('./train.csv')
# temp_df['Cabin'].isna().head()
cnt=0
for b in temp_df['Cabin'].isna():
    cnt+= (b==True)
cnt

687

In [2]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
combine = [train_df, test_df]
PassengerId = test_df['PassengerId']

for dataset in combine:
    #Pclass
    dataset['upperClass'] = np.where(dataset['Pclass']==1,1,0)
    dataset['middleClass'] = np.where(dataset['Pclass']==2,1,0)
    dataset['lowerClass'] = np.where(dataset['Pclass']==3,1,0)
    #Title
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Mr'] = np.where(dataset['Title']=='Mr',1,0)
    dataset['Miss'] = np.where(dataset['Title']=='Miss',1,0)
    dataset['Mrs'] = np.where(dataset['Title']=='Mrs',1,0)
    dataset['Master'] = np.where(dataset['Title']=='Master',1,0)
    dataset['rareTitle'] = np.where(dataset['Title']=='Rare',1,0)
    #Gender
    dataset['female'] = np.where(dataset['Sex']=='female',1,0)
    dataset['male'] = np.where(dataset['Sex']=='male',1,0)
    #Cabin
    dataset['CabinChar'] = dataset['Cabin'].str[:1]
    dataset['A'] = np.where(dataset['CabinChar']=='A',1,0)
    dataset['B'] = np.where(dataset['CabinChar']=='B',1,0)
    dataset['C'] = np.where(dataset['CabinChar']=='C',1,0)
    dataset['D'] = np.where(dataset['CabinChar']=='D',1,0)
    dataset['E'] = np.where(dataset['CabinChar']=='E',1,0)
    dataset['noCabin'] = np.where(dataset['Cabin'].isnull(),1,0)
    #Embarked
    dataset['Cherbourg'] = np.where(dataset['Embarked']=='C',1,0)
    dataset['Queenstown'] = np.where(dataset['Embarked']=='Q',1,0)
    dataset['Southampton'] = np.where(dataset['Embarked']=='S',1,0)
    #No Family
    dataset['noFamily'] = np.where(dataset['SibSp'] + dataset['Parch']==0,1,0)
    dataset['familySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    dataset['hasChildren'] = np.where((dataset['Age'][0]>=25 and dataset['Parch'][0]>=1), 1, 0)
    
    #Fare (filled below with log)
    dataset['payingGreat'] = np.where(dataset['Fare']> dataset['Fare'].mean(), 1, 0)
    
    #Age (filled below with log)
    dataset['isOlderThanAverage'] = np.where(dataset['Age']> dataset['Age'].mean(), 1, 0)
    dataset['isMinor'] = np.where(('Age' in dataset.columns and dataset['Age']<18), 1, 0);
        
    
# train_df['Survived'] = train_df['Survived'].replace(0,-1)
    
# Average age on Title
all_df = pd.concat([train_df, test_df])
ageGroup = all_df.groupby('Title')['Age'].mean()
# Ticket information
ticketSize = all_df.groupby('Ticket')['PassengerId'].count()

for dataset in combine:
#     if('Survived' in dataset.columns and dataset['Survived'][0]==0):
#         continue
    # Ticket Size, Ticket Survived %
    dataset['ticketSize'] = dataset['Ticket'].map(ticketSize)
    dataset['noTicketPartner'] = np.where(dataset['ticketSize']==1,1,0)
    # Null
    dataset['noAge'] = np.where(dataset['Age'].isnull(),1,0)
    dataset['Age'] = dataset['Age'].fillna(dataset['Title'].map(ageGroup))
    dataset['Fare'] = dataset['Fare'].fillna(dataset['Fare'].mean())
    # Log
#     dataset['Age'] = np.where(dataset['Age'] < 1, 1, dataset['Age'])
#     dataset['Age'] = np.log(dataset['Age'])
#     dataset['Fare'] = np.where(dataset['Fare'] < 1, 1, dataset['Fare'])
#     dataset['Fare'] = np.log(dataset['Fare'])
    
"""
Need to calculate the reference surived rate group by ticket
Calculation need to exclude himself, otherwise the field is already have the survive data
"""
ticketInTrain = train_df.groupby('Ticket')['PassengerId'].count()
ticketSurvived = train_df.groupby('Ticket')['Survived'].sum()/ train_df.groupby('Ticket')['PassengerId'].count()


train_df['noTicketRef'] = np.where(train_df['Ticket'].map(ticketInTrain)==1,1,0)
test_df['noTicketRef'] = np.where(test_df['Ticket'].map(ticketInTrain)>0,0,1)
# train_df['ticketRef'] = np.where(train_df['noTicketRef']==1,0
#         ,(train_df['Ticket'].map(ticketSurvived) * train_df['Ticket'].map(ticketInTrain) - train_df['Survived'])/(train_df['Ticket'].map(ticketInTrain) - 1))
# test_df['ticketRef'] = np.where(test_df['noTicketRef']==1,0,test_df['Ticket'].map(ticketSurvived))

train_result = train_df['Survived']
train_df.to_csv('all.csv', index=False)
#Drop
train_df = train_df.drop(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Title', 'Ticket', 'Embarked', 'CabinChar', 'Cabin', 'noTicketRef'], axis=1)
test_df = test_df.drop(['PassengerId', 'Pclass', 'Name', 'Title', 'Sex', 'Ticket', 'Embarked', 'CabinChar', 'Cabin', 'noTicketRef'], axis=1)

train_df.head()

/home/pp/softwares/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:56: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



,Age,SibSp,Parch,Fare,upperClass,middleClass,lowerClass,Mr,Miss,Mrs,...,Southampton,noFamily,familySize,hasChildren,payingGreat,isOlderThanAverage,isMinor,ticketSize,noTicketPartner,noAge
0,22.0,1,0,7.2500,0,0,1,1,0,0,...,1,0,2,0,0,0,0,1,1,0
1,38.0,1,0,71.2833,1,0,0,0,0,1,...,0,0,2,0,1,1,0,2,0,0
2,26.0,0,0,7.9250,0,0,1,0,1,0,...,1,1,1,0,0,0,0,1,1,0
3,35.0,1,0,53.1000,1,0,0,0,0,1,...,1,0,2,0,1,1,0,2,0,0
4,35.0,0,0,8.0500,0,0,1,1,0,0,...,1,1,1,0,0,1,0,1,1,0


In [3]:
# ticketInTrain
temp_df = pd.read_csv('./train.csv')
# temp_df.groupby('Ticket')['Ticket'].count()
# temp_df['Ticket'].map(ticketInTrain)

**Using Keras build neural network**

-1st layer nodes = number of input * 2 (dropout 0.5)

-2nd layer nodes = 1st layer nodes/2

-Negative output means dead, Positive output means alive

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from math import exp
from keras.layers import LeakyReLU

/home/pp/softwares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# create some data
train_df = train_df.values
train_result = train_result.values
test_df = test_df.values

x = train_df
y = train_result
z = test_df

In [6]:
print(x.shape)
# sum(y==0)
# xx = list(list())
# yy = list()
# for i in y:
#     if i==0:
#         yy.append(i)
#         xx.append(x[i])
# x = np.array(xx)
# y = np.array(yy)
# sum(y==0)

(891, 32)


In [7]:
from sklearn import preprocessing

# x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
# df = pandas.DataFrame(x_scaled)
x= x_scaled

In [8]:

# build a neural network from the 1st layer to the last layer
model = Sequential()
model.add(Dense(units=128, input_dim=32))
# model.add(Dropout(0.25)
model.add(Dense(units=256, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5))
model.add(Dense(units=512, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5))
model.add(Dense(units=1024, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5))
model.add(Dense(units=512, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5))
model.add(Dense(units=256, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.5))
model.add(Dense(units=128, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.25))
model.add(Dense(units=64, activation=LeakyReLU(alpha=0.3))) 
# model.add(Dropout(0.1))
model.add(Dense(units=1, activation='sigmoid'))

/home/pp/softwares/anaconda3/lib/python3.6/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


In [9]:

# choose loss function and optimizing method
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

In [10]:



# # training
# print('Training -----------')
# for step in range(8001):
#     cost = model.train_on_batch(x, y)
#     if step % 1000 == 0:
#         print('step', step, 'train cost:', cost)
        
# print("----TRAINING FINISHED---")


In [11]:
model.fit(x, y,
          batch_size=int(len(x)/10),
          epochs=1000,
          verbose=2, validation_split=0.2, shuffle=True)

Train on 712 samples, validate on 179 samples
Epoch 1/1000
 - 1s - loss: 0.2501 - acc: 0.4831 - val_loss: 0.2486 - val_acc: 0.5866
Epoch 2/1000
 - 0s - loss: 0.2467 - acc: 0.6699 - val_loss: 0.2448 - val_acc: 0.6704
Epoch 3/1000
 - 0s - loss: 0.2439 - acc: 0.6362 - val_loss: 0.2416 - val_acc: 0.6313
Epoch 4/1000
 - 0s - loss: 0.2416 - acc: 0.6180 - val_loss: 0.2387 - val_acc: 0.6480
Epoch 5/1000
 - 0s - loss: 0.2394 - acc: 0.6110 - val_loss: 0.2362 - val_acc: 0.6425
Epoch 6/1000
 - 0s - loss: 0.2377 - acc: 0.6096 - val_loss: 0.2341 - val_acc: 0.6425
Epoch 7/1000
 - 0s - loss: 0.2361 - acc: 0.6096 - val_loss: 0.2322 - val_acc: 0.6425
Epoch 8/1000
 - 0s - loss: 0.2347 - acc: 0.6096 - val_loss: 0.2305 - val_acc: 0.6425
Epoch 9/1000
 - 0s - loss: 0.2334 - acc: 0.6096 - val_loss: 0.2289 - val_acc: 0.6425
Epoch 10/1000
 - 0s - loss: 0.2321 - acc: 0.6096 - val_loss: 0.2273 - val_acc: 0.6425
Epoch 11/1000
 - 0s - loss: 0.2308 - acc: 0.6096 - val_loss: 0.2256 - val_acc: 0.6425
Epoch 12/1000
 - 

Epoch 96/1000
 - 0s - loss: 0.1416 - acc: 0.8174 - val_loss: 0.1215 - val_acc: 0.8324
Epoch 97/1000
 - 0s - loss: 0.1412 - acc: 0.8188 - val_loss: 0.1211 - val_acc: 0.8324
Epoch 98/1000
 - 0s - loss: 0.1410 - acc: 0.8230 - val_loss: 0.1208 - val_acc: 0.8324
Epoch 99/1000
 - 0s - loss: 0.1407 - acc: 0.8202 - val_loss: 0.1204 - val_acc: 0.8324
Epoch 100/1000
 - 0s - loss: 0.1405 - acc: 0.8272 - val_loss: 0.1201 - val_acc: 0.8324
Epoch 101/1000
 - 0s - loss: 0.1403 - acc: 0.8230 - val_loss: 0.1197 - val_acc: 0.8324
Epoch 102/1000
 - 0s - loss: 0.1400 - acc: 0.8230 - val_loss: 0.1194 - val_acc: 0.8324
Epoch 103/1000
 - 0s - loss: 0.1397 - acc: 0.8244 - val_loss: 0.1191 - val_acc: 0.8324
Epoch 104/1000
 - 0s - loss: 0.1394 - acc: 0.8244 - val_loss: 0.1188 - val_acc: 0.8324
Epoch 105/1000
 - 0s - loss: 0.1393 - acc: 0.8244 - val_loss: 0.1186 - val_acc: 0.8324
Epoch 106/1000
 - 0s - loss: 0.1391 - acc: 0.8216 - val_loss: 0.1182 - val_acc: 0.8324
Epoch 107/1000
 - 0s - loss: 0.1388 - acc: 0.82

Epoch 191/1000
 - 0s - loss: 0.1291 - acc: 0.8244 - val_loss: 0.1048 - val_acc: 0.8547
Epoch 192/1000
 - 0s - loss: 0.1290 - acc: 0.8301 - val_loss: 0.1047 - val_acc: 0.8659
Epoch 193/1000
 - 0s - loss: 0.1289 - acc: 0.8301 - val_loss: 0.1046 - val_acc: 0.8659
Epoch 194/1000
 - 0s - loss: 0.1289 - acc: 0.8272 - val_loss: 0.1046 - val_acc: 0.8547
Epoch 195/1000
 - 0s - loss: 0.1289 - acc: 0.8329 - val_loss: 0.1042 - val_acc: 0.8603
Epoch 196/1000
 - 0s - loss: 0.1288 - acc: 0.8315 - val_loss: 0.1042 - val_acc: 0.8603
Epoch 197/1000
 - 0s - loss: 0.1287 - acc: 0.8258 - val_loss: 0.1042 - val_acc: 0.8603
Epoch 198/1000
 - 0s - loss: 0.1286 - acc: 0.8272 - val_loss: 0.1042 - val_acc: 0.8547
Epoch 199/1000
 - 0s - loss: 0.1286 - acc: 0.8258 - val_loss: 0.1041 - val_acc: 0.8547
Epoch 200/1000
 - 0s - loss: 0.1288 - acc: 0.8287 - val_loss: 0.1039 - val_acc: 0.8603
Epoch 201/1000
 - 0s - loss: 0.1285 - acc: 0.8244 - val_loss: 0.1038 - val_acc: 0.8603
Epoch 202/1000
 - 0s - loss: 0.1284 - acc: 

 - 0s - loss: 0.1238 - acc: 0.8399 - val_loss: 0.0981 - val_acc: 0.8547
Epoch 286/1000
 - 0s - loss: 0.1237 - acc: 0.8329 - val_loss: 0.0981 - val_acc: 0.8547
Epoch 287/1000
 - 0s - loss: 0.1240 - acc: 0.8329 - val_loss: 0.0979 - val_acc: 0.8547
Epoch 288/1000
 - 0s - loss: 0.1239 - acc: 0.8343 - val_loss: 0.0978 - val_acc: 0.8547
Epoch 289/1000
 - 0s - loss: 0.1236 - acc: 0.8329 - val_loss: 0.0976 - val_acc: 0.8659
Epoch 290/1000
 - 0s - loss: 0.1239 - acc: 0.8258 - val_loss: 0.0980 - val_acc: 0.8547
Epoch 291/1000
 - 0s - loss: 0.1239 - acc: 0.8329 - val_loss: 0.0982 - val_acc: 0.8547
Epoch 292/1000
 - 0s - loss: 0.1235 - acc: 0.8385 - val_loss: 0.0975 - val_acc: 0.8659
Epoch 293/1000
 - 0s - loss: 0.1234 - acc: 0.8329 - val_loss: 0.0975 - val_acc: 0.8603
Epoch 294/1000
 - 0s - loss: 0.1233 - acc: 0.8329 - val_loss: 0.0975 - val_acc: 0.8547
Epoch 295/1000
 - 0s - loss: 0.1234 - acc: 0.8329 - val_loss: 0.0975 - val_acc: 0.8547
Epoch 296/1000
 - 0s - loss: 0.1232 - acc: 0.8357 - val_lo

Epoch 380/1000
 - 0s - loss: 0.1194 - acc: 0.8511 - val_loss: 0.0951 - val_acc: 0.8771
Epoch 381/1000
 - 0s - loss: 0.1195 - acc: 0.8441 - val_loss: 0.0942 - val_acc: 0.8827
Epoch 382/1000
 - 0s - loss: 0.1192 - acc: 0.8525 - val_loss: 0.0939 - val_acc: 0.8939
Epoch 383/1000
 - 0s - loss: 0.1193 - acc: 0.8469 - val_loss: 0.0947 - val_acc: 0.8827
Epoch 384/1000
 - 0s - loss: 0.1191 - acc: 0.8497 - val_loss: 0.0944 - val_acc: 0.8827
Epoch 385/1000
 - 0s - loss: 0.1193 - acc: 0.8469 - val_loss: 0.0942 - val_acc: 0.8827
Epoch 386/1000
 - 0s - loss: 0.1194 - acc: 0.8455 - val_loss: 0.0939 - val_acc: 0.8883
Epoch 387/1000
 - 0s - loss: 0.1194 - acc: 0.8483 - val_loss: 0.0947 - val_acc: 0.8827
Epoch 388/1000
 - 0s - loss: 0.1194 - acc: 0.8455 - val_loss: 0.0941 - val_acc: 0.8827
Epoch 389/1000
 - 0s - loss: 0.1191 - acc: 0.8483 - val_loss: 0.0943 - val_acc: 0.8883
Epoch 390/1000
 - 0s - loss: 0.1193 - acc: 0.8497 - val_loss: 0.0939 - val_acc: 0.8827
Epoch 391/1000
 - 0s - loss: 0.1189 - acc: 

...

Epoch 569/1000
 - 0s - loss: 0.1119 - acc: 0.8666 - val_loss: 0.0924 - val_acc: 0.8771
Epoch 570/1000
 - 0s - loss: 0.1115 - acc: 0.8638 - val_loss: 0.0929 - val_acc: 0.8827
Epoch 571/1000
 - 0s - loss: 0.1115 - acc: 0.8567 - val_loss: 0.0920 - val_acc: 0.8715
Epoch 572/1000
 - 0s - loss: 0.1119 - acc: 0.8581 - val_loss: 0.0924 - val_acc: 0.8771
Epoch 573/1000
 - 0s - loss: 0.1119 - acc: 0.8666 - val_loss: 0.0929 - val_acc: 0.8827
Epoch 574/1000
 - 0s - loss: 0.1117 - acc: 0.8610 - val_loss: 0.0923 - val_acc: 0.8771
Epoch 575/1000
 - 0s - loss: 0.1117 - acc: 0.8596 - val_loss: 0.0929 - val_acc: 0.8827
Epoch 576/1000
 - 0s - loss: 0.1115 - acc: 0.8539 - val_loss: 0.0919 - val_acc: 0.8771
Epoch 577/1000
 - 0s - loss: 0.1121 - acc: 0.8567 - val_loss: 0.0925 - val_acc: 0.8827
Epoch 578/1000
 - 0s - loss: 0.1116 - acc: 0.8596 - val_loss: 0.0928 - val_acc: 0.8827
Epoch 579/1000
 - 0s - loss: 0.1111 - acc: 0.8610 - val_loss: 0.0937 - val_acc: 0.8827
Epoch 580/1000
 - 0s - loss: 0.1111 - acc: 

 - 0s - loss: 0.1084 - acc: 0.8638 - val_loss: 0.0933 - val_acc: 0.8827
Epoch 664/1000
 - 0s - loss: 0.1082 - acc: 0.8694 - val_loss: 0.0921 - val_acc: 0.8715
Epoch 665/1000
 - 0s - loss: 0.1080 - acc: 0.8666 - val_loss: 0.0923 - val_acc: 0.8715
Epoch 666/1000
 - 0s - loss: 0.1081 - acc: 0.8694 - val_loss: 0.0945 - val_acc: 0.8883
Epoch 667/1000
 - 0s - loss: 0.1077 - acc: 0.8680 - val_loss: 0.0937 - val_acc: 0.8827
Epoch 668/1000
 - 0s - loss: 0.1078 - acc: 0.8694 - val_loss: 0.0926 - val_acc: 0.8827
Epoch 669/1000
 - 0s - loss: 0.1078 - acc: 0.8694 - val_loss: 0.0948 - val_acc: 0.8883
Epoch 670/1000
 - 0s - loss: 0.1082 - acc: 0.8722 - val_loss: 0.0957 - val_acc: 0.8883
Epoch 671/1000
 - 0s - loss: 0.1084 - acc: 0.8652 - val_loss: 0.0930 - val_acc: 0.8827
Epoch 672/1000
 - 0s - loss: 0.1078 - acc: 0.8694 - val_loss: 0.0926 - val_acc: 0.8827
Epoch 673/1000
 - 0s - loss: 0.1074 - acc: 0.8694 - val_loss: 0.0932 - val_acc: 0.8827
Epoch 674/1000
 - 0s - loss: 0.1073 - acc: 0.8694 - val_lo

Epoch 758/1000
 - 0s - loss: 0.1044 - acc: 0.8806 - val_loss: 0.0946 - val_acc: 0.8883
Epoch 759/1000
 - 0s - loss: 0.1043 - acc: 0.8764 - val_loss: 0.0926 - val_acc: 0.8771
Epoch 760/1000
 - 0s - loss: 0.1053 - acc: 0.8722 - val_loss: 0.0930 - val_acc: 0.8827
Epoch 761/1000
 - 0s - loss: 0.1039 - acc: 0.8764 - val_loss: 0.0931 - val_acc: 0.8827
Epoch 762/1000
 - 0s - loss: 0.1043 - acc: 0.8736 - val_loss: 0.0924 - val_acc: 0.8715
Epoch 763/1000
 - 0s - loss: 0.1053 - acc: 0.8750 - val_loss: 0.0943 - val_acc: 0.8883
Epoch 764/1000
 - 0s - loss: 0.1037 - acc: 0.8750 - val_loss: 0.0959 - val_acc: 0.8883
Epoch 765/1000
 - 0s - loss: 0.1036 - acc: 0.8792 - val_loss: 0.0932 - val_acc: 0.8827
Epoch 766/1000
 - 0s - loss: 0.1036 - acc: 0.8792 - val_loss: 0.0935 - val_acc: 0.8827
Epoch 767/1000
 - 0s - loss: 0.1041 - acc: 0.8750 - val_loss: 0.0944 - val_acc: 0.8883
Epoch 768/1000
 - 0s - loss: 0.1036 - acc: 0.8792 - val_loss: 0.0941 - val_acc: 0.8883
Epoch 769/1000
 - 0s - loss: 0.1039 - acc: 

 - 0s - loss: 0.0999 - acc: 0.8792 - val_loss: 0.0950 - val_acc: 0.8827
Epoch 853/1000
 - 0s - loss: 0.1012 - acc: 0.8778 - val_loss: 0.0952 - val_acc: 0.8827
Epoch 854/1000
 - 0s - loss: 0.1011 - acc: 0.8848 - val_loss: 0.0981 - val_acc: 0.8827
Epoch 855/1000
 - 0s - loss: 0.0991 - acc: 0.8862 - val_loss: 0.0950 - val_acc: 0.8827
Epoch 856/1000
 - 0s - loss: 0.1009 - acc: 0.8778 - val_loss: 0.0946 - val_acc: 0.8827
Epoch 857/1000
 - 0s - loss: 0.0997 - acc: 0.8848 - val_loss: 0.0965 - val_acc: 0.8827
Epoch 858/1000
 - 0s - loss: 0.0998 - acc: 0.8778 - val_loss: 0.0959 - val_acc: 0.8827
Epoch 859/1000
 - 0s - loss: 0.0996 - acc: 0.8862 - val_loss: 0.0972 - val_acc: 0.8827
Epoch 860/1000
 - 0s - loss: 0.1004 - acc: 0.8834 - val_loss: 0.0983 - val_acc: 0.8827
Epoch 861/1000
 - 0s - loss: 0.1009 - acc: 0.8792 - val_loss: 0.0951 - val_acc: 0.8827
Epoch 862/1000
 - 0s - loss: 0.1002 - acc: 0.8834 - val_loss: 0.0947 - val_acc: 0.8771
Epoch 863/1000
 - 0s - loss: 0.0989 - acc: 0.8876 - val_lo

Epoch 947/1000
 - 0s - loss: 0.0965 - acc: 0.8862 - val_loss: 0.0959 - val_acc: 0.8827
Epoch 948/1000
 - 0s - loss: 0.0973 - acc: 0.8862 - val_loss: 0.0994 - val_acc: 0.8883
Epoch 949/1000
 - 0s - loss: 0.0979 - acc: 0.8834 - val_loss: 0.0976 - val_acc: 0.8883
Epoch 950/1000
 - 0s - loss: 0.0972 - acc: 0.8862 - val_loss: 0.0963 - val_acc: 0.8771
Epoch 951/1000
 - 0s - loss: 0.0970 - acc: 0.8848 - val_loss: 0.0982 - val_acc: 0.8883
Epoch 952/1000
 - 0s - loss: 0.0965 - acc: 0.8862 - val_loss: 0.0969 - val_acc: 0.8771
Epoch 953/1000
 - 0s - loss: 0.0966 - acc: 0.8862 - val_loss: 0.0975 - val_acc: 0.8827
Epoch 954/1000
 - 0s - loss: 0.0960 - acc: 0.8876 - val_loss: 0.0978 - val_acc: 0.8883
Epoch 955/1000
 - 0s - loss: 0.0955 - acc: 0.8919 - val_loss: 0.0981 - val_acc: 0.8883
Epoch 956/1000
 - 0s - loss: 0.0960 - acc: 0.8919 - val_loss: 0.0988 - val_acc: 0.8883
Epoch 957/1000
 - 0s - loss: 0.0986 - acc: 0.8778 - val_loss: 0.0971 - val_acc: 0.8771
Epoch 958/1000
 - 0s - loss: 0.0960 - acc: 

In [12]:

# predict
test_predict = model.predict(z)
# exp(test_predict.ravel()).mean()
# temp = list(test_predict.flatten())
# exp(temp).mean()
sum=0
for x in test_predict:
    sum+= exp(x)
print(sum/ test_predict.shape[0])

1.367348796412067


In [17]:
# Generate Submission File
test_predict = np.where(test_predict>.5,1,0)
NNSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': test_predict.ravel() })
NNSubmission.to_csv("NNSubmission.csv", index=False)

In [18]:
test_predict[:100]

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
    